In [ ]:
pwd

### for Jupyter JSC

In [ ]:
#!module load Stages/2022
#!module load GCC
#!module load Python/3.9.6

import os
os.environ['PROJECT_DIR']=os.path.expandvars( "$PROJECT/OSCBAGDIS" )
os.environ['OSCBAGDIS_DATAPROC_CODE']=os.path.expandvars( "$PROJECT/OSCBAGDIS/data_proc_code" )
os.environ['CODE']=os.path.expandvars( "$PROJECT/OSCBAGDIS/data_proc_code" )
os.environ['DATA_DUSS']=os.path.expandvars( "$PROJECT_DIR/data_proc_code" )
#os.environ['CODE_MEMORY_ERRORS'] = os.path.expandvars("$PROJECT/lyon/memerr/code" )
sys.path.append( os.path.expandvars( "$OSCBAGDIS_DATAPROC_CODE" ) )

# Autoreload setup

In [ ]:
%load_ext autoreload
%autoreload 2
import globvars as gv
from globvars import gp

In [ ]:
#$RS --mods msrc --feat_types H_act --parcel_group_names Sensorimotor,OccipitalI    nf,FrontalSup,FrontalInf,TemporalMid  --prefix onlyH_act_SMyOIyFSyFIyTM_noLFP

# Set params

In [ ]:
inc_S03 = True
inc_noMvt = False
join_medcond = False  # medconds merged
join_subjects = False
test_best_LFP_disjointness = False
#subskips = [4,8] # 1, 4 ,8
subskips = [8] # 1, 4 ,8
#disjoint_windows = False  # when it is False it just means we are not forcing it, rather using param file setup
#overlapping_windows = False
#if disjoint_windows == overlapping_windows and overlapping_windows == True:
#    raise ValueError('not implemented, do only one for now')
# first is cortex side, second is LFP side
#side_tuples = [ ('both','both') ]
#side_tuples = [ ('left','left'), ('right','right') ]
side_tuples = [ ('both','both') ]
#side_tuples += [ ('left','left'), ('right','right') ]
side_tuples += [ ('left_exCB','left'), ('right_exCB','right') ]
side_tuples += [ ('left_onlyCB','left'),  ('both_onlyCB','left'), 
                ('right_onlyCB','right'),  ('both_onlyCB','right') ]
#('right_onlyCB','left'), ('left_onlyCB','right'),


rawnames_dict = {}
prefixes_types = []
#prefixes_types = ['PREFIXES_H_ACT_PER_PARCEL_GROUP_LFP', 'PREFIXES_H_ACT_PER_PARCEL_GROUP', "PREFIXES_MAIN"]
prefixes_types = ['PREFIXES_MAIN']
#prefixes_types = ['PREFIXES_H_ACT_PER_PARCEL_GROUP']
#prefixes_types = ['PREFIXES_H_ACT_PER_PARCEL_GROUP_LFP']
prefixes_types = ['PREFIXES_H_ACT_PER_PARCEL_GROUP', 'PREFIXES_H_ACT_PER_PARCEL_GROUP_LFP']  # _LFP here means 'and' LFP
# not "SIDED" version selects parcel groups on both brain sides
prefixes_types = ['PREFIXES_SPECIFIC_SIDED', 'PREFIXES_H_ACT_PER_PARCEL_GROUP_SIDED', 
                  'PREFIXES_H_ACT_PER_PARCEL_GROUP_SIDED_LFP'] # _LFP here means 'and' LFP
#prefixes_types = ['PREFIXES_SPECIFIC']
#prefixes_types = ['PREFIXES_SPECIFIC_SIDED'] + prefixes_types
# to check
#prefixes_types = ['PREFIXES_SPECIFIC_SIDED']
# last
prefixes_types += ['PREFIXES_MAIN']
prefixes_types += ['PREFIXES_SPECIFIC_SIDED']
prefixes_types += ['PREFIXES_H_ACT_PER_PARCEL_GROUP_SIDED', 
                   'PREFIXES_H_ACT_PER_PARCEL_GROUP_SIDED_LFP']

#prefixes_types = []  # aug 22
#prefixes_types += ['PREFIXES_H_SEP']
# TODO include in final

#fname_runstr = '_runstrings_ML_test.txt'
fname_runstr = '_runstrings_ML.txt'
g_it_strs = []
# g_it_strs += [' --groupings_to_use merge_nothing   --int_types_to_use basic']
# g_it_strs += [' --groupings_to_use merge_movements --int_types_to_use basic']
# g_it_strs += [' --groupings_to_use merge_nothing --int_types_to_use trem_vs_quiet']
g_it_strs += [' --groupings_to_use merge_all_not_trem --int_types_to_use basic']
#modes = ['only', 'exclude'] 
modes = ['only']  # exclude is boring anyway, other parcels compensate too much

#comment = 'Per subject brain area significance both sides (separately), test which best LFP is better: disjoint or not -- max-overlap'
comment = 'Per subject brain area significance both sides (separately), disjoint and semi-disjoint'

runpars = {}
runpars['inc_S03'] = inc_S03
runpars['join_subjects'] =join_subjects
runpars['inc_noMvt'] = inc_noMvt
runpars['join_medcond'] = join_medcond
#
#runpars['inc_disjoint_windows'] = disjoint_windows
#runpars['inc_overlapping_windows'] = overlapping_windows
runpars['side_tuples'] = side_tuples
runpars['prefixes_types'] = prefixes_types
runpars['fname_runstr'] = fname_runstr
runpars['barain_area_inclusion_modes'] = modes
#runpars['test_best_LFP_disjointness'] = test_best_LFP_disjointness
runpars['subskips'] = subskips
runpars['run_specific_args'] = ' --output_subdir per_subj_per_medcond_best_LFP_wholectx'  #techincal params about inclusion of some algortihms or no
#runpars['run_specific_args'] = ' --subskip_fit 4 --output_subdir per_subj_per_medcond_best_LFP_wholectx_subskip4 --bestLFP_disjoint 1'  #techincal params about inclusion of some algortihms or no
#runpars['run_specific_args'] = ' --subskip_fit 1 --output_subdir per_subj_per_medcond_best_LFP_wholectx_subskip1' 
runpars['comment'] = comment
print('Setting params finished')

# Apply params

In [ ]:
rawnames_dict['off_Mvt_best'] = ['S01_off_hold',
 'S01_off_move',
 'S02_off_hold',
 'S02_off_move',
 'S04_off_hold',
 'S04_off_move',
 'S05_off_hold',
 'S05_off_move',
 'S07_off_hold',
 'S07_off_move']
rawnames_dict['off_unsure'] = ['S03_off_hold', 'S03_off_move']
rawnames_dict['off_Mvt'] = rawnames_dict['off_Mvt_best'] + rawnames_dict['off_unsure']
rawnames_dict['on_Mvt'] = ['S01_on_hold',
 'S01_on_move',
 'S02_on_hold',
 'S02_on_move',
 'S04_on_hold',
 'S04_on_move',
 'S05_on_hold',
 'S05_on_move',
 'S07_on_hold',
 'S07_on_move'] 

rns = rawnames_dict['off_Mvt_best'] + rawnames_dict['on_Mvt']
z = zip(rns[::2], rns[1::2])
per_subj_tasks_merged = list(z)
per_subj_tasks_merged = [ rawnames_dict['off_unsure'] ] + per_subj_tasks_merged
##############################################
rawnames_list = []
if join_subjects:
    if not join_medcond:
        if inc_S03:
            rawnames_list += [ ('normal', rawnames_dict['off_Mvt'] ) ] 
        else:
            rawnames_list += [ ('normal', rawnames_dict['off_Mvt_best'] ) ] 
        if inc_noMvt:
            raise ValueError('not implemented')

        rawnames_list += [ ('normal',rawnames_dict['on_Mvt']) ] 

        if inc_noMvt:
            raise ValueError('not implemented')
    else:
        if inc_S03:
            kk = 'off_Mvt'
        else:
            kk = 'off_Mvt_best'
        rns = rawnames_dict[kk] + rawnames_dict['on_Mvt']

        rawnames_list += [ ('normal',rns ) ] 
else:
    if not join_medcond:
        for tpl in per_subj_tasks_merged:
            rawnames_list += [ ('normal', list(tpl) ) ] 
    else:
        raise ValueError('Not implemented yet')

##############################################
if join_subjects:
    if join_medcond:
        rn_test =  ('test',[ 'S01_off_hold', 'S04_off_hold', 'S05_off_move'] ) 
    else:
        rn_test =  ('test',[ 'S01_off_hold', 'S01_on_move', 'S04_off_hold', 'S05_on_move'] ) 
else:
    if not join_medcond:
        rn_test =  ('test',[ 'S04_off_hold', 'S04_off_move'] ) 
    else:
        rn_test =  ('test',[ 'S04_off_hold', 'S04_on_move'] ) 
rawnames_list += [rn_test]
##############################################
pfstr_per_rnt = {}

if join_subjects:
    if join_medcond:
        pfile_str = ' --param_file ML_joint2_one_LFP_HPC.ini'
        #TODO: create a new pfile for testing, this one does not exit
        pfile_str_test = ' --param_file ML_joint2_one_LFP_HPC_fast.ini'
    else:
        pfile_str = ' --param_file ML_joint_one_LFP_HPC.ini'
        pfile_str_test = ' --param_file ML_joint_one_LFP_HPC_fast.ini' 
else:
    if not join_medcond:
        #pfile_str      = ' --param_file ML_medcondsep_one_LFP_HPC.ini'
        #pfile_str_test = ' --param_file ML_medcondsep_one_LFP_HPC_fast.ini' 
        pfile_str      = ' --param_file ML_medcondsep_one_LFP_wholectx_HPC.ini'
        pfile_str_test = ' --param_file ML_medcondsep_one_LFP_wholectx_HPC_fast.ini' 
    
pfstr_per_rnt ['normal'] = pfile_str
pfstr_per_rnt ['test']   = pfile_str_test

#############################################
BANDS_BETA = 'beta'
BANDS_GAMMA = 'gamma'
BANDS_TREMOR = 'tremor'
rslist_cur = []

if "PREFIXES_MAIN" in prefixes_types:
    rslist_cur += [('modLFP',              '--mods LFP')]
    rslist_cur += [('onlyH_act',           '--feat_types H_act')]        
    rslist_cur += [('onlyH',               '--feat_types H_act,H_mob,H_compl')]        
    rslist_cur += [('modSrc_self',         '--self_couplings_only 1 --mods msrc')]
    rslist_cur += [('LFPrel_noself',       '--LFP_related_only 1  --cross_couplings_only 1')] 
    rslist_cur += [('allb_beta_noH',       f'--fbands {BANDS_BETA}  --feat_types con,rbcorr')]
    rslist_cur += [('allb_gamma_noH',      f'--fbands {BANDS_GAMMA}   --feat_types con,rbcorr')]
    rslist_cur += [('allb_tremor_noH',    f'--fbands {BANDS_TREMOR}  --feat_types con,rbcorr')]
    rslist_cur += [('modSrc',              '--mods msrc')]   
    
if "PREFIXES_SPECIFIC" in prefixes_types:
    rslist_cur += [('modLFP',              '--mods LFP')]
    rslist_cur += [('onlyH_act',           '--feat_types H_act')]    
    
if "PREFIXES_H_SEP" in prefixes_types:
    rslist_cur += [('onlyH_mob',             '--feat_types H_mob')]        
    rslist_cur += [('onlyH_compl',           '--feat_types H_compl')]   
    
if "PREFIXES_SPECIFIC_SIDED" in prefixes_types:
    # note that I don't want modLFP because I don't compute other feature types besides Hjorth
    if test_best_LFP_disjointness:
        rslist_cur += [('onlyH_act_bestLFPdisj',              '--feat_types H_act  --bestLFP_disjoint 1')]        
        rslist_cur += [('onlyH_bestLFPdisj',                  '--feat_types H_act,H_mob,H_compl  --bestLFP_disjoint 1')]        
        rslist_cur += [('onlyH_act_bestLFPoverlap',           '--feat_types H_act  --bestLFP_disjoint 0')]        
        rslist_cur += [('onlyH_bestLFPoverlap',               '--feat_types H_act,H_mob,H_compl  --bestLFP_disjoint 0')]        
    else:
#        rslist_cur += [('onlyH_act',           '--feat_types H_act')]        
#        rslist_cur += [('onlyH',               '--feat_types H_act,H_mob,H_compl')]        
        rslist_cur += [('onlyH_modLFP',        '--mods LFP --feat_types H_act')]
        rslist_cur += [('onlyH_act_modLFP',    '--mods LFP --feat_types H_act,H_mob,H_compl')]
        
        rslist_cur += [('onlyH_mob',             '--feat_types H_mob')]        
        rslist_cur += [('onlyH_compl',           '--feat_types H_compl')]        
    
    #rslist_cur += ['--feat_types H_act --mods LFP      --prefix onlyH_act_onlyLFP         ']        
    #rslist_cur += ['--feat_types H_act --mods LFP      --prefix onlyH_act_onlyLFP         ']   


rslist_cur2 = []
for subskip in subskips:
    for pref,argstr in rslist_cur:
        argstr2 = f'{argstr} --subskip_fit {subskip} --bestLFP_disjoint auto'
        pref2 = f'{pref}_subskip{subskip}'
        rslist_cur2 += [ (pref2,argstr2) ]
rslist_cur = rslist_cur2
    
#--subskip_fit 4 --output_subdir per_subj_per_medcond_best_LFP_wholectx_subskip4 --bestLFP_disjoint 1

display(rawnames_list, prefixes_types, pfstr_per_rnt, g_it_strs, rslist_cur )

assert len(rslist_cur) == len(set(rslist_cur))  # I don't want reapeats
prefs = list( list( zip(*rslist_cur) )[0] )
assert len(prefs) == len(set(prefs)), prefs  # I don't want reapeats even in prefs

In [ ]:
print( test_best_LFP_disjointness )

In [ ]:
r = set(list(range(256 ))) - \
set([0,1,2,5,6,9,10,13,14,17,18,21,22,25,26,29,30,33,34,35,36,37,38,41,42,43,45,46,47,49,50,53,54,55,56,57,58,59,60,61,62,65,66,69,70,73,74,77,78,79,80,81,82,83,84,85,86,89,90,93,94,97,98,99,101,102,103,105,106,107,108,109,110,113,114,117,118,121,122,125,126,127,128,129,130,131,132,133,134,137,138,141,142,145,146,149,150,151,152,153,154,155,156,157,158,161,162,165,166,169,170,171,172,173,174,175,176,177,178,179,180,181,182,185,186,187,188,189,190,193,194,197,198,199,200,201,202,203,204,205,206,209,210,213,214,217,218,221,222,223,224,225,226,227,229,230,233,234,237,238,241,242,245,246,247,249,250,253,254])
','.join( map(str, list(r) )) 

In [ ]:
p = [100,104,11,11,111,112,115,115,116,116,119,12,12,120,123,123,124,135,135,136,139,140,143,144,147,148,15,159,159,16,160,160,163,164,167,168,183,184,19,191,192,195,196,20,207,208,211,212,215,216,219,220,228,23,231,232,235,236,239,24,240,243,244,248,251,252,255,27,28,3,31,32,39,4,40,44,48,51,52,63,64,67,68,7,71,72,75,75,76,8,87,88,91,92,95,96]
#-------------------- prefixes COMPLETED, N=176
c = [0,1,2,5,6,9,10,13,14,17,18,21,22,25,26,29,30,33,34,35,36,37,38,41,42,43,45,46,47,49,50,53,54,55,56,57,58,59,60,61,62,65,66,69,70,73,74,77,78,79,80,81,82,83,84,85,86,89,90,93,94,97,98,99,101,102,103,105,106,107,108,109,110,113,114,117,118,121,122,125,126,127,128,129,130,131,132,133,134,137,138,141,142,145,146,149,150,151,152,153,154,155,156,157,158,161,162,165,166,169,170,171,172,173,174,175,176,177,178,179,180,181,182,185,186,187,188,189,190,191,193,194,195,197,198,199,200,201,202,203,204,205,206,209,210,213,214,217,218,219,221,222,223,224,225,226,227,228,229,230,233,234,237,238,241,242,243,245,246,247,248,249,250,251,253,254]
#----
lst = list( set(c) - set(p) ) 
print(  ','.join( map(str,lst))   )

# Set timestamp

In [ ]:
import time
import datetime
print(f'Set new timestamp at {datetime.datetime.now()}')
run_corresp_id = int( time.time() * 10000 )


# Generate file

In [ ]:
rawnames_list,prefixes_types,g_it_strs,modes,runpars,run_corresp_id,pfstr_per_rnt,rslist_cur, side_tuples

In [ ]:
save = True
from utils_runstrings_gen_HPC import genRunstringsML
runstrings,runstrings_per_rnt,pref2pgn = \
    genRunstringsML(rawnames_list,prefixes_types,g_it_strs,modes,runpars,
               run_corresp_id,pfstr_per_rnt,rslist_cur, side_tuples)
n_testrunstr_to_include = 1
    
if n_testrunstr_to_include > 0:
    runstrings = runstrings_per_rnt['test'][:n_testrunstr_to_include] + runstrings

srs = set(runstrings)
assert len(srs) == len(runstrings),  f'there are repeating runstrings {len(srs)} , {len(runstrings)}'

####################
        
from os.path import join as pjoin
import json

if save:
    fname_full_runstr = pjoin(gv.code_dir, 'run', fname_runstr)
    with open( fname_full_runstr, 'w' ) as f:
        for s in runstrings:
            f.write(s + '\n')
    fname_full_runstr_archive = pjoin(gv.code_dir, 'run', f'__{fname_runstr[:-4]}_{run_corresp_id}.json')
    with open( fname_full_runstr, 'w' ) as f:
        for s in runstrings:
            f.write(s + '\n')
            
    print(fname_full_runstr)
    print(fname_full_runstr_archive)

c = {'correspondance':pref2pgn, 'prefixes_types':prefixes_types, 
         'param_dir_str':pfstr_per_rnt ['normal'], 'rawnames_list':rawnames_list,
        'comment':comment, 'runpars':runpars, 'runstrings':runstrings}
if save:
    fn = pjoin(gv.code_dir, 'run', f'___run_corresp_{run_corresp_id}.txt')    
    with open(fn , 'w' ) as f:
        json.dump(c, f, indent =2)
#     f.write(json.dumps(pref2pgn))

    print(fn)
    print(f'Saved res for run_corresp_id = {run_corresp_id}')

In [ ]:
len(runstrings)

In [ ]:
from collections import Counter
cts = Counter(runstrings)
#for rs in runstrings:
#    rs.count()
cts

In [ ]:
onlyH_modLFP_subskip8BB, both

In [ ]:
runstrings

In [ ]:
#len(runstrings)

In [ ]:
inds=[661,663,665,667,669,671,673,675,677,679,681,683,685,687,689,691,693,695,697,699,701,703]
rs = np.array(runstrings)[inds]
rs.tolist()

In [ ]:
prefixes_types

## find inds to recalc

In [ ]:
to_recalc = [('S04_on',   'onlyH_act_only22') ]



rsis = []
for rn,name in to_recalc:
    key_founds = []
    for key,item in pref2pgn.items():
        if item[1] == name:
            key_founds += [key]
    if not len(key_founds) and (name in pref2pgn):
        key_founds = [name]
        roi = pref2pgn[name][1]
    else:
        roi = name
    print('key_founds = ', key_founds)
        
    for rsi,rs in enumerate(runstrings):
        if rs.find('_test') >= 0 or rs.find('_fast') >= 0:
            continue
        items = rs.split()
        a = items[1::2]
        b = items[2::2]
        d = dict( zip(a,b) )
        
        c1 = d['-r'].find(rn) >= 0 
        c2 = d['--prefix'] in key_founds
        
        if c1 and c2:
            rsis += [rsi]
            print(f'{rsi:4} = {rn} : {roi:18} -> {d["--prefix"]}')
            #print(rsi)
            
rsis = list(sorted(set(rsis)))
print('\nto be given to sbatch: ', rsis)

In [ ]:
264 - 256

In [ ]:
import gc; gc.collect()

In [ ]:
key_found

In [ ]:
pref2pgn

In [ ]:
pref2pgn

In [ ]:
runpars

In [ ]:
pref2pgn

In [ ]:
#prefixes_types, pfstr_per_rnt ['normal']

In [ ]:
runstrings

In [ ]:
pref2pgn

In [ ]:
special_globinds

In [ ]:
len(runstrings)

In [ ]:
#special_globinds_mod = special_globinds
special_globinds_mod = [gi + n_testrunstr_to_include for gi in special_globinds]
#[runstrings[gi][120:-93] for gi in special_globinds_mod]
runstrings_sub = [runstrings[gi] for gi in special_globinds_mod]
display( runstrings_sub )
#special_globinds_mod
print( ','.join(map(str,special_globinds_mod) ) )

In [ ]:
len(runstrings)

In [ ]:
sis = []
for rsi,rs in enumerate(runstrings):
    #if rs.find('--mods LFP') >= 0:
    if rs.find('15') >= 0:
        sis += [rsi]
        print(rs)
print( ','.join(map(str,sis) ) )

In [ ]:
corresp

In [ ]:
runstrings_per_rnt['test' ]

In [ ]:
runstrings_per_rnt

In [ ]:
gv.data_dir

In [ ]:
pwd

In [ ]:
pref2pgn

In [ ]:
runstrings[0]

In [ ]:

#open()

In [ ]:
s